In [15]:
import face_recognition
import numpy as np
import glob
import cv2

ModuleNotFoundError: No module named 'cv2'

make an array of all the saved jpg files' paths

In [4]:
known_face_encodings = []
known_face_names = []

list_of_files = [f for f in glob.glob('known_people/*.jpg')]
number_files = len(list_of_files)

names = list_of_files.copy()

for i in range(number_files):
    globals()['image_{}'.format(i)] = face_recognition.load_image_file(list_of_files[i])
    globals()['image_encoding_{}'.format(i)] = face_recognition.face_encodings(globals()['image_{}'.format(i)])[0]
    known_face_encodings.append(globals()['image_encoding_{}'.format(i)])

    # Create array of known names
    names[i] = names[i].replace("known_people/", "")  
    known_face_names.append(names[i])

In [8]:
image_to_recognize = face_recognition.load_image_file('images_to_recognize/bundesratsfoto_2018.png')

In [9]:
face_locations = []
face_encodings = []
face_names = []

In [11]:
face_locations = face_recognition.face_locations(image_to_recognize)
face_encodings = face_recognition.face_encodings(image_to_recognize, face_locations)

face_names = []
for face_encoding in face_encodings:
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
    name = "Unknown"

    # # If a match was found in known_face_encodings, just use the first one.
    # if True in matches:
    #     first_match_index = matches.index(True)
    #     name = known_face_names[first_match_index]

    # Or instead, use the known face with the smallest distance to the new face
    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = known_face_names[best_match_index]

    face_names.append(name)

In [12]:
face_names

['Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'alain_berset.jpg',
 'Unknown',
 'Unknown',
 'Unknown']

In [13]:
face_locations

[(252, 1317, 295, 1274),
 (242, 1183, 286, 1140),
 (246, 317, 282, 281),
 (209, 170, 252, 127),
 (218, 674, 262, 631),
 (238, 1005, 274, 969),
 (247, 477, 290, 434),
 (254, 865, 290, 829)]

[array([-0.10206273,  0.04238825,  0.0263379 ,  0.01901536, -0.11224031,
        -0.05027467,  0.01602385, -0.07758858,  0.12967065, -0.07952845,
         0.24524319, -0.07463314, -0.22791059, -0.08783046, -0.05306987,
         0.10160114, -0.16351944, -0.03689432, -0.20076205, -0.05166239,
        -0.03107794,  0.03623536,  0.10611191, -0.03896081, -0.12661767,
        -0.42955267, -0.09251368, -0.09952834,  0.06326776, -0.08017639,
         0.05246459,  0.07780933, -0.24359258, -0.15837705,  0.0023041 ,
        -0.05135359, -0.05031474,  0.00118915,  0.21813796, -0.01367772,
        -0.2130941 ,  0.09901219,  0.1285859 ,  0.23157881,  0.18378377,
         0.04873853,  0.01317706, -0.07246675,  0.06555182, -0.31678265,
         0.023298  ,  0.15724874,  0.11142813,  0.01297295,  0.12706728,
        -0.02810281,  0.03416705,  0.11867051, -0.17921576,  0.11177244,
         0.14190963, -0.01615351, -0.01855488,  0.04404593,  0.11743889,
         0.02600626, -0.09025133, -0.05936651,  0.1